In [1]:
#%%capture
#pip install import_ipynb
#import import_ipynb

#from Formalize import data_pre_lockdown, data_post_lockdown

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import re
# from mlutils import dataset, connector
import scipy.stats
from scipy.stats import *
from sklearn.preprocessing import StandardScaler
import math
import matplotlib.pyplot as plt
import warnings
import statsmodels.api as sm 
import seaborn as sns
import pylab as py 
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
class All_Data():
    
    def __init__(self, file):
        self.obj = pd.read_csv(file)

    def visualize(self):
        print('self.obj: ',format(self.obj))
        
    def drop_columns(self, column_names, axis):
        self.obj = self.obj.drop(column_names, axis = axis)
        return self

In [4]:
class Retailers():
    
    def __init__(self, inventory_limit, signature):
        self.signature = signature
        self.preferences = {}
        self.requirement = {}
        self.predict_orders = pd.DataFrame(columns = ['Product_id', 'month', 'week', 'Warehouse_Name', 'Order_Units', 'distance', 'Additional_Costs'])
        self.inventory_limit = int(inventory_limit)
        self.inventory = {1: int(self.inventory_limit / 6), 2: int(self.inventory_limit / 6), 3: int(self.inventory_limit / 6), 4: int(self.inventory_limit / 6), 5: int(self.inventory_limit / 6), 6: int(self.inventory_limit / 6)}
        self.assumed_demand = {1: 29, 2: 29, 3: 29, 4: 29, 5: 29, 6: 29}
        self.actual_demand = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}
        self.historic_demand = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}
        self.prev_demand_fulfilled = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}
        
        print('New retailer successfully added.')
        
    def Extract_Data(self, All_Data_obj, column, value):
        self.obj = All_Data_obj.loc[All_Data_obj[column] == value]
        self.obj = self.obj.reset_index()
        return self
        
    def visualize(self):
        print('self.obj: ',format(self.obj))
        
    def drop_columns(self, column_names, axis):
        self.obj = self.obj.drop(column_names, axis = axis)
        return self
    
    def Update_History(self):
        n = len(self.history)
        for i in range(n):
            mean = np.mean(self.history[i]['store_cost_sum'])
            self.history[i]['mean_sell_price'] = mean
            sd = np.std(self.history[i]['store_cost_sum'])
            self.history[i]['sd_sell_price'] = sd
        return self
    
    def Add_Preference(self, name, key, value):
        self.preferences.update({key: (name, value)})
        self.preferences = dict(sorted(self.preferences.items()))
        return self
    
    def Additional_Costs(self, cost_price, miles, warehouse_size, order_size = 33):
        if order_size == 0:
            return 0
        else:
            alpha = 0.261
            beta = 0.261
            gamma = 0.00105
            a = cost_price * alpha
            b = (miles * beta) / order_size
            c = (warehouse_size * gamma) / order_size
            return a + b + c
    
    def curate(self, total, minim, maxim):
        A = []
        for x in range(min(total + 1, maxim + 1)):
            if x <= minim - 1:
                x = 0
            #print('x: ',format(x))
            for y in range(min((total + 1 - x), maxim + 1)):
                if y <= minim - 1:
                    y = 0
                #print('y: ',format(y))
                z = min((total - x - y), maxim)
                if z < minim:
                    z = 0
                #print('total - x - y: ',format(total - x - y))
                #print('maxim: ',format(maxim))
                #print('z: ',format(z))
                A.append((x, y, z))
        return A

    def Analyze_Additional_Costs(self, cost_price, x, y, z):
        M = list(self.preferences.keys())
        S_1 = ((self.preferences[M[0]])[1]).size
        S_2 = ((self.preferences[M[1]])[1]).size
        S_3 = ((self.preferences[M[2]])[1]).size
        a = self.Additional_Costs(cost_price, M[0], S_1, x)
        b = self.Additional_Costs(cost_price, M[1], S_2, y)
        c = self.Additional_Costs(cost_price, M[2], S_3, z)
        w = a * x + b * y + c * z
        return round(w, 4)

    def Predict_Order_Levels(self, month, week):
        
        for i in range(month - 1,71,12):
            h = self.history[i]
            product_id = int((i / 12) + 1)
            cost_price = np.mean(h['cost_price'])
            t = h[h['week'] == week]
            T = list(t['weekly_demand'])[0]
            self.historic_demand[product_id] = T
            
        if (list(self.actual_demand.values()) == list([0] * 6)):
            alpha = 0.5
            beta = 0.5
            gamma = 0
        else:
            alpha = 0.34
            beta = 0.33
            gamma = 0.33
                
        for i in range(6):
            total = alpha * self.historic_demand[i + 1] + beta * self.assumed_demand[i + 1] + gamma * self.actual_demand[i + 1]
            
            total = int(total)
            
            A = self.curate(total = total, minim = 5, maxim = 33)
            X = []
            Y = []
            Z = []
            W = []
            for (x, y, z) in A:
                if (x + y + z) == total:
                    X.append(x)
                    Y.append(y)
                    Z.append(z)
                    w = self.Analyze_Additional_Costs(cost_price, x, y, z)
                    W.append(w)
            if W == []:
                continue
                #print('W: ',format(W))
                #print('A: ',format(A))
            else:
                W.reverse()
                k = W.index(min(W))
                M = list(self.preferences.keys())
                V = list(self.preferences.values())
                print('self.predict_orders: ')
                print(self.predict_orders)
                
                self.predict_orders = self.predict_orders.append({'Product_id': i + 1, 'month': month, 'week': week, 'Warehouse_Name': V[0][0], 'Order_Units': X[-k - 1], 'distance': M[0], 'Additional_Costs': self.Analyze_Additional_Costs(cost_price, X[-k - 1], 0, 0)}, ignore_index = True)
                self.predict_orders = self.predict_orders.append({'Product_id': i + 1, 'month': month, 'week': week, 'Warehouse_Name': V[1][0], 'Order_Units': Y[-k - 1], 'distance': M[1], 'Additional_Costs': self.Analyze_Additional_Costs(cost_price, 0, Y[-k - 1], 0)}, ignore_index = True)
                self.predict_orders = self.predict_orders.append({'Product_id': i + 1, 'month': month, 'week': week, 'Warehouse_Name': V[2][0], 'Order_Units': Z[-k - 1], 'distance': M[2], 'Additional_Costs': self.Analyze_Additional_Costs(cost_price, 0, 0, Z[-k - 1])}, ignore_index = True)
                self.requirement.update({i + 1: X[-k - 1] + Y[-k - 1] + Z[-k - 1]})
                #print('self.requirement: ')
                #print(self.requirement)
        return self

In [5]:
class Product_Wise_Data(Retailers):
    
    def __init__(self, retailer, column = 'product_id'):
        self.retailer = retailer
        self.retailer.products = retailer.obj[column].unique()
        print('Class Initialized.')
        
    def Extract_Data(self, column, product_id):
        df = self.retailer.obj.loc[self.retailer.obj[column] == product_id]
        df = df.reset_index()
        return df
        
    def visualize(self):
        print('self.obj: ',format(self.retailer.obj))
        
    def drop_columns(self, df, column_names, axis):
        df = df.drop(column_names, axis = axis)
        return df
    
    def Magic_Touch(self, column = 'product_id', verbose = False):
        self.retailer.product = []
        for i in range(len(self.retailer.products)):
            df = self.Extract_Data(column, self.retailer.products[i])
            df = self.drop_columns(df, 'index', axis = 1)
            self.retailer.product.append(df)
        if verbose == True:
            print('self.retailer.product: ')
            print(self.retailer.product)
        return self.retailer

In [6]:
class History(Retailers):
    
    def __init__(self, retailer, column = 'time_month'):
        self.retailer = retailer
        self.retailer.timeline = retailer.obj[column].unique()
        print('Class Initialized.')
        
    def Extract_Data(self, df, column, product_id):
        df = df.loc[df[column] == product_id]
        df = df.reset_index(drop = True)
        return df
        
    def visualize(self):
        print('self.obj: ',format(self.retailer.obj))
        
    def Magic_Touch(self, column = 'time_month', verbose = False):
        self.retailer.history = []
        for i in range(len(self.retailer.product)):
            for j in range(len(self.retailer.timeline)):
                df = self.retailer.product[i]
                #print('df: ')
                #print(df)
                df = self.Extract_Data(df, column, self.retailer.timeline[j])
                self.retailer.history.append(df)
        if verbose == True:
            print('self.retailer.history: ')
            print(self.retailer.history)
        return self.retailer
    
    def Add_Latest_Demand(self, df, month, week, column = 'product_id', verbose = False):
        df = df.loc[(df['time_month'] == month) & (df['week'] == week)]
        #print('df: ')
        #print(df)
        self = self.New_Data(df, column)
        return self.retailer
    
    def New_Data(self, df, column = 'product_id', verbose = False):
        y = self.retailer.signature + '.new_history'
        if y not in globals():
            self.retailer.new_history = [[]]
        else:
            self.retailer.new_history.append([])
        primary = len(self.retailer.new_history)
        #print('primary: ',format(primary))
        for i in range(len(self.retailer.product)):
            df_1 = df.loc[df['retailer'] == int((self.retailer.signature)[-1])]
            df_1 = self.Extract_Data(df_1, column, i + 1)
            self.retailer.new_history[primary - 1].append(df_1)
        if verbose == True:
            print('self.retailer.new_history: ')
            print(self.retailer.new_history)
        return self

In [7]:
class Warehouse():
    
    def __init__(self, size, max_order_limit, signature):
        self.signature = signature
        self.size = size
        self.max_order_limit = max_order_limit
        self.product_limit = int((self.size) / 6)
        self.availability = []
        for i in range(6):
            self.availability.append(self.product_limit)
        print('Warehouse created succesfully.')

In [8]:
class Region():
    
    def __init__(self, signature):
        self.signature = signature
        self.retailers = []
        self.pipeline_i = pd.DataFrame(columns = ['Product_id', 'Retailer', 'month', 'week', 'Warehouse_Name', 'Order_Units', 'distance', 'Additional_Costs'])
        self.warehouses = []
        self.accept_orders = pd.DataFrame(columns = ['Product_id', 'Retailer', 'month', 'week', 'Warehouse_Name', 'Order_Units', 'distance', 'Additional_Costs'])
        self.excess = {'Warehouse_1': 0, 'Warehouse_2': 0, 'Warehouse_3': 0}
        self.required = {'Warehouse_1': 0, 'Warehouse_2': 0, 'Warehouse_3': 0}
        self.local_demand = {'Warehouse_1': 0, 'Warehouse_2': 0, 'Warehouse_3': 0}
        self.global_flags = {'Warehouse_1': 0, 'Warehouse_2': 0, 'Warehouse_3': 0}
        print('Region created successfully.')
        
    def Add_Retailer(self, Retailer):
        self.retailers.append(Retailer)
        print('Retailer added successfully.')
        return self
    
    def Add_Warehouse(self, Warehouse):
        self.warehouses.append(Warehouse)
        print('Warehouse added successfully.')
        return self
    
    def Level_1(self, product_id):
        frames = []
        for r in self.retailers:
            x = r.predict_orders
            #print('x: ')
            #print(x)
            K = x.loc[x['Product_id'] == product_id]
            K['Retailer'] = r.signature
            cols = K.columns.to_list()
            cols = cols[:1] + cols[-1:] + cols[1:-1]
            K = K[cols]
            frames.append(K)
        self.accept_orders = pd.concat(frames)
        return self
        
    def Level_2(self, product_id):
        C = self.accept_orders.loc[self.accept_orders['Warehouse_Name'] == 'Warehouse_1']
        self.local_demand.update({'Warehouse_1': sum(C['Order_Units'])})
        C = self.accept_orders.loc[self.accept_orders['Warehouse_Name'] == 'Warehouse_2']
        self.local_demand.update({'Warehouse_2': sum(C['Order_Units'])})
        C = self.accept_orders.loc[self.accept_orders['Warehouse_Name'] == 'Warehouse_3']
        self.local_demand.update({'Warehouse_3': sum(C['Order_Units'])})
        
        for w in list(self.local_demand.keys()):
            if w == self.warehouses[0].signature:
                if self.local_demand[w] > self.warehouses[0].availability[product_id - 1]:
                    self.required[w] = self.local_demand[w] - self.warehouses[0].availability[product_id - 1]
                if self.local_demand[w] < self.warehouses[0].availability[product_id - 1]:
                    self.excess[w] = self.warehouses[0].availability[product_id - 1] - self.local_demand[w]
            else:
                self.required[w] = self.local_demand[w]
        return self
        
    def Level_3(self):
        frames = []
        L = list(self.global_flags.keys())
        #print('self.accept_orders: ')
        #print(self.accept_orders)
        for l in list(L):
            K = self.accept_orders.loc[self.accept_orders['Warehouse_Name'] == l].sort_values(by = 'distance')
            frames.append(K)
        self.accept_orders = pd.concat(frames)
        self.accept_orders = self.accept_orders.reset_index(drop = True)
        return self
    
    def Level_4(self, product_id):
        dummy_warehouse_availability = self.warehouses[0].availability[product_id - 1]
        for i in range(len(self.retailers) - 1, -1, -1):
            print('self.accept_orders: ')
            print(self.accept_orders)
            p = self.accept_orders.loc[i, 'Retailer']
            q = self.accept_orders.loc[(self.accept_orders['Retailer'] == p) & (self.accept_orders['Warehouse_Name'] == self.warehouses[0].signature)]
            if (dummy_warehouse_availability >= (list(q['Order_Units']))[0]):
                self.pipeline_i = self.pipeline_i.append(q, ignore_index = True)
                dummy_warehouse_availability = dummy_warehouse_availability - (list(q['Order_Units']))[0]
            else:
                ix = q.index
                q.at[ix, 'Order_Units'] = dummy_warehouse_availability
                self.pipeline_i = self.pipeline_i.append(q, ignore_index = True)
                dummy_warehouse_availability = 0
                break
        return self
    
    def Evaluate_I(self, product_id):
        self = self.Level_1(product_id)
        self = self.Level_2(product_id)
        return self
    
    def Evaluate_II(self, product_id):
        self = self.Level_3()
        self = self.Level_4(product_id)
        return self
    
    def Execute_Pipeline_I(self, product_id):
        i = 0
        while len(self.pipeline_i.Product_id):
            q = self.pipeline_i.loc[i]
            p = q['Retailer']
            self.warehouses[0].availability[product_id - 1] = self.warehouses[0].availability[product_id - 1] - q['Order_Units']
            for r in self.retailers:
                if p == r.signature:
                    #r.prev_demand_fulfilled = r.inventory
                    ix = r.predict_orders.index[(r.predict_orders['Product_id'] == product_id) & (r.predict_orders['Warehouse_Name'] == self.warehouses[0].signature)].tolist()
                    r.predict_orders.at[ix[0], 'Order_Units'] = r.predict_orders.at[ix[0], 'Order_Units'] - q['Order_Units']
                    #print('r.requirement: ')
                    #print(r.requirement)
                    r.requirement[product_id] = r.requirement[product_id] - q['Order_Units']
                    r.inventory[product_id] = r.inventory[product_id] + q['Order_Units']
                    break
            self.pipeline_i = self.pipeline_i.drop([0])
            self.pipeline_i = self.pipeline_i.reset_index(drop = True)
        self = self.Level_1(product_id)
        return self
    

In [9]:
class Central():
    
    def __init__(self):
        self.regions = []
        self.excess = {'Warehouse_1': 0, 'Warehouse_2': 0, 'Warehouse_3': 0}
        self.pipeline_ii = pd.DataFrame(columns = ['Product_id', 'Retailer', 'month', 'week', 'Warehouse_Name', 'Order_Units', 'distance'])
        print('Central system successfully initialized.')
        
    def Add_Region(self, Region):
        self.regions.append(Region)
        print('Region added successfully.')
        return self
    
    def Analyze_I(self):
        self.excess = {'Warehouse_1': 0, 'Warehouse_2': 0, 'Warehouse_3': 0}
        for r in self.regions:
            #print('r.signature: ',format(r.signature))
            #print('self.excess: ',format(self.excess))
            self.excess['Warehouse_1'] = self.excess['Warehouse_1'] + r.excess['Warehouse_1']
            self.excess['Warehouse_2'] = self.excess['Warehouse_2'] + r.excess['Warehouse_2']
            self.excess['Warehouse_3'] = self.excess['Warehouse_3'] + r.excess['Warehouse_3']
        self.excess = dict(sorted(self.excess.items(), key=lambda item: item[1], reverse = True))
        #print('self.excess: ',format(self.excess))
        for r in self.regions:
            r.global_flags = self.excess
        return self
    
    
    def Analyze_II(self, product_id):
        self.pipeline_ii = pd.DataFrame(columns = ['Product_id', 'Retailer', 'month', 'week', 'Warehouse_Name', 'Order_Units', 'distance'])
        for i in self.regions:
            P = i.retailers
            for p in P:
                if p.requirement[product_id] > 0:
                    #print("p.signature: ",format(p.signature))
                    #print("list(self.excess.values())[0]: ",format(list(self.excess.keys())[0]))
                    q = i.accept_orders.loc[(i.accept_orders['Product_id'] == product_id) & (i.accept_orders['Retailer'] == p.signature) & (i.accept_orders['Warehouse_Name'] == list(self.excess.keys())[0])]
                    q = q.drop('Additional_Costs', axis = 1)
                    #print("q: ")
                    #print(q)
                    ix = q.index
                    q.at[ix, 'Order_Units'] = p.requirement[product_id]
                    self.pipeline_ii = self.pipeline_ii.append(q, ignore_index = True)
                    self.pipeline_ii = self.pipeline_ii.sort_values(by = 'distance')
        return self
    
    def Execute_Pipeline_II(self, product_id):
        dummy_warehouse_signature = list(self.excess.keys())[0]
        for r in self.regions:
            if r.warehouses[0].signature == dummy_warehouse_signature:
                Warehouse = r.warehouses[0]
                break
        
        i = 0
        while len(self.pipeline_ii.Product_id):
            q = self.pipeline_ii.loc[i]
            #print('q: ')
            #print(q)
            p = q['Retailer']
            if (Warehouse.availability[product_id - 1] >= q['Order_Units']):
                Warehouse.availability[product_id - 1] = Warehouse.availability[product_id - 1] - q['Order_Units']
            else:
                ix = q.index
                q['Order_Units'] = Warehouse.availability[product_id - 1]
                Warehouse.availability[product_id - 1] = 0
                #print('q: ')
                #print(q)
            for reg in self.regions:
                for r in reg.retailers:
                    if p == r.signature:
                        ix = r.predict_orders.index[(r.predict_orders['Product_id'] == 1) & (r.predict_orders['Warehouse_Name'] == Warehouse.signature)].tolist()
                        r.predict_orders.at[ix[0], 'Order_Units'] = r.predict_orders.at[ix[0], 'Order_Units'] - q['Order_Units']
                        r.requirement[product_id] = r.requirement[product_id] - q['Order_Units']
                        r.inventory[product_id] = r.inventory[product_id] + q['Order_Units']
                        break
                else:
                    continue
                break
            if Warehouse.availability[product_id - 1] == 0:
                self.pipeline_ii.at[0, 'Order_Units'] = r.requirement[product_id]
                break
            else:
                self.pipeline_ii = self.pipeline_ii.drop([0])
                self.pipeline_ii = self.pipeline_ii.reset_index(drop = True)
            
        return self
    

In [10]:
def standarise(column,pct,pct_lower):
    sc = StandardScaler() 
    y = column.to_list()
    #print('y: ',format(y))
    y.sort()
    #print('y: ',format(y))
    len_y = len(y)
    y = y[int(pct_lower * len_y):int(len_y * pct)]
    len_y = len(y)
    yy=([[x] for x in y])
    sc.fit(yy)
    y_std =sc.transform(yy)
    y_std = y_std.flatten()
    return y_std,len_y,y

In [11]:
def find_distribution(column,pct,pct_lower):
    # Set up list of candidate distributions to use
    # See https://docs.scipy.org/doc/scipy/reference/stats.html for more
    y_std,size,y_org = standarise(column,pct,pct_lower)
    dist_names = ['norm','beta',
                 'invgauss','uniform','expon', 'lognorm','triang']

    chi_square_statistics = []
    
    # 11 bins
    percentile_bins = np.linspace(0,100,11)
    percentile_cutoffs = np.percentile(y_std, percentile_bins)
    #print('sorted(percentile_cutoffs): ',format(sorted(percentile_cutoffs)))
    #print('sorted(y_std): ',format(sorted(y_std)))
    observed_frequency, bins = (np.histogram(sorted(y_std), bins=sorted(percentile_cutoffs)))
    cum_observed_frequency = np.cumsum(observed_frequency)

    # Loop through candidate distributions

    for distribution in dist_names:
        # Set up distribution and get fitted distribution parameters
        dist = getattr(scipy.stats, distribution)
        param = dist.fit(y_std)
        #print("{}\n{}\n".format(dist, param))
      

        # Get expected counts in percentile bins
        # cdf of fitted sistrinution across bins
        cdf_fitted = dist.cdf(percentile_cutoffs, *param)
        expected_frequency = []
        for bin in range(len(percentile_bins)-1):
            expected_cdf_area = cdf_fitted[bin+1] - cdf_fitted[bin]
            expected_frequency.append(expected_cdf_area)

        # Chi-square Statistics
        expected_frequency = np.array(expected_frequency) * size
        cum_expected_frequency = np.cumsum(expected_frequency)
        #print('cum_expected_frequency: ',format(cum_expected_frequency))
        #print('cum_observed_frequency: ',format(cum_observed_frequency))
   
        #print('type(cum_expected_frequency): ',format(type(cum_expected_frequency)))
        #print('type(cum_observed_frequency): ',format(type(cum_observed_frequency)))
        
        #print('0 in cum_expected_frequency: ',format(0 in cum_expected_frequency))
        #print('0 in cum_observed_frequency: ',format(0 in cum_observed_frequency))
        
        if 0 in cum_observed_frequency:
            add = 0
            for i in range(len(cum_observed_frequency)):
                if cum_observed_frequency[i] != 0:
                    add = add + ((cum_expected_frequency[i] - cum_observed_frequency[i]) ** 2) / cum_observed_frequency[i]
            ss = round(add,0)
        else:
            ss = round(sum (((cum_expected_frequency - cum_observed_frequency) ** 2) / cum_observed_frequency),0)
        
        #ss = sum (((cum_expected_frequency - cum_observed_frequency) ** 2) / cum_observed_frequency),0
        chi_square_statistics.append(ss)


    #Sort by minimum ch-square statistics
    results = pd.DataFrame()
    results['Distribution'] = dist_names
    results['chi_square'] = chi_square_statistics
    results.sort_values(['chi_square'], inplace=True)

    results = results.reset_index(drop = True)
    
    #print('type(results): ',format(type(results)))
    D = results.loc[:,'Distribution']
    print("D[0]: ",format(D[0]))
    e = D[0]
    
    #print ('\nDistributions listed by Betterment of fit:')
    #print ('............................................')
    #print (results)
    return e

In [12]:
def fit_distribution(L, column):

    B = []
    E = []    
    dist_names = ['norm','beta',
                 'invgauss','uniform','expon', 'lognorm','triang']

    for i in range(6):
        A = L[i][column]
        #l = np.std(A)
        #SD.update({i:l})
        #print(i)
        #plt.hist(A)
        #plt.show()
    
        e = find_distribution(A,0.99,0.01)
        E.append(e)
        #fit_distribution(A,0.99,0.01)
    
        y_std,len_y,y = standarise(A,0.99,0.01)
    
        plt.hist(y)
        plt.xlabel('A[%i]' %i)
        plt.ylabel('Frequency')
    
        plt.show()
    
        location = min(y)
        #print('location: ',format(location))
        scale = max(y) - location
        #print('scale: ',format(scale))
        mean = np.mean(y)
        #print('mean: ',format(mean))
        std = np.std(y)
        #print('std: ',format(std))
        mean_mod = (mean - location) / scale
        #print('mean_mod: ',format(mean_mod))
        std_mod = std / scale
        #print('std_mod: ',format(std_mod))
        para_a = mean_mod * (((mean_mod * (1 - mean_mod)) / (std_mod) ** 2) - 1)
        #print('para_a: ',format(para_a))
        para_b = (1 - mean_mod) * (((mean_mod * (1 - mean_mod)) / (std_mod) ** 2) - 1)
        #print('para_b: ',format(para_b))
        
        if e == dist_names[0]:
            plt.plot(y,norm.pdf(y_std, para_b, para_a)) # , para_a, para_b, -3 * location, scale
    
            plt.plot(y,norm.pdf(y_std, 2.94, 1.51)) # 1.51, 2.94, -1.71, 5.02
            plt.xlabel('Demand (Units)\n\nNormal Distribution')
    
            plt.show()
        
        elif e == dist_names[1]:
            plt.plot(y,beta.pdf(y_std, para_a, para_b, -3 * location, scale))
    
            plt.plot(y,beta.pdf(y_std,1.51, 2.94, -1.71, 5.02))
            plt.xlabel('Demand (Units)\n\nBeta Distribution')
    
            plt.show()
        
        elif e == dist_names[2]:
            plt.plot(y,invgauss.pdf(y_std, para_a, -3 * location, scale)) # , para_b
    
            plt.plot(y,invgauss.pdf(y_std,1.51, -1.71, 5.02)) # 1.51, 2.94, -1.71, 5.02
            # 0.45, -1.64, 3.61
            
            plt.xlabel('Demand (Units)\n\nInverse-Gaussian Distribution')
    
            plt.show()
        
        elif e == dist_names[3]:
            plt.plot(y,uniform.pdf(y_std, para_a, para_b, -3 * location, scale))
    
            plt.plot(y,uniform.pdf(y_std,1.51, 2.94, -1.71, 5.02))
            plt.xlabel('Demand (Units)\n\nUniform Distribution')
    
            plt.show()
        
        elif e == dist_names[4]:
            plt.plot(y,expon.pdf(y_std, -3 * location, scale)) # para_a, para_b, 
    
            plt.plot(y,expon.pdf(y_std,-1.71, 5.02)) # 1.51, 2.94, -1.71, 5.02
            # -1.19, 1.19
            
            plt.xlabel('Demand (Units)\n\nExponential Distribution')
    
            plt.show()
        
        elif e == dist_names[5]:
            plt.plot(y,lognorm.pdf(y_std, para_a, -3 * location, scale)) # , para_b
    
            plt.plot(y,lognorm.pdf(y_std, 1.51, -1.71, 5.02)) # 1.51, 2.94, -1.71, 5.02
        
            plt.xlabel('Demand (Units)\n\nLog-Normal Distribution')
    
            plt.show()
        
        elif e == dist_names[6]:
            plt.plot(y,triang.pdf(y_std, para_a, -3 * location, scale)) #  para_b,
    
            plt.plot(y,triang.pdf(y_std,1.51, -1.71, 5.02)) # 1.51, 2.94, -1.71, 5.02
            # 0.12, -1.79, 4.90
        
            plt.xlabel('Demand (Units)\n\nTriangular Distribution')
    
            plt.show()
    
        B.append(A)